# Further Hypothesis Testing

In [ ]:
# Select this cell and type Ctrl-Enter to execute the code below.

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt


## 1 - Introduction

In this workshop, you will apply a number of commonly encountered parametric and non-parametric tests to answer a variety of research questions about an example data set.

We begin with a brief exploration of the data.

### Data set

The file `stars.csv` contains a dataset of 240 stars, with five variables for each star:

|variable | description|
|:---|:---|
|temperature | the surface temperature (K)|
|luminosity | luminosity relative to sun|
|radius | radius relative to sun|
|spectral_class | the spectral class of each star (O,B,A,F,G,K or M)|
|type | as defined below|


The luminosity and radius of each star is calculated relative to that of the Sun:

$L_{sun} = 3.83 \times 10^{26}\text{W}$

$R_{sun} = 6.96 \times 10^8\text{m}$


The stars are classified into 6 types:

code | type
:---|:---
0 |Brown Dwarf
1 |Red Dwarf
2 |White Dwarf
3 |Main Sequence
4 |Supergiant
5 |Hypergiant

The dataset contains 40 examples of each type.


Load the data using the `pandas` library:

In [ ]:
data = pd.read_csv("stars.csv")
type_key = ['Brown Dwarf', 'Red Dwarf', 'White Dwarf', 'Main Sequence', 'Supergiant','Hypergiant']

data.head()

We will be using histogram plots from `matplotlib` to visualise distributions of these variables.

For example, execute the following to see an overall histogram of luminosity:

In [ ]:
sample = data.luminosity
xlab = 'luminosity'
ylab = 'freq'

bins = np.linspace(sample.min(), sample.max())
ax = plt.axes()
ax.set_xlabel(xlab)
ax.set_ylabel(ylab)
plt.hist(sample, bins, alpha=0.5, color='gray')
plt.show()

Or, more usefully, on a log scale:

In [ ]:
sample = data.luminosity.apply(np.log)
xlab = 'log(luminosity)'
ylab = 'freq'

bins = np.linspace(sample.min(), sample.max())
ax = plt.axes()
ax.set_xlabel(xlab)
ax.set_ylabel(ylab)
plt.hist(sample, bins, alpha=0.5, color='gray')
plt.show()

We can split the data by another variable to compare different groups of stars. 

For example, the following shows log(luminosity), grouped by type:

In [ ]:
sample = data.luminosity.apply(np.log)
grouped = sample.groupby(data.type)
xlab = 'log(luminosity)'
ylab = 'freq'

bins = np.linspace(sample.min(), sample.max())
ax = plt.axes()
ax.set_xlabel(xlab)
ax.set_ylabel(ylab)
grouped.apply(lambda x: plt.hist(x, bins, alpha=0.5, color = 'C' + str(x.name), label=type_key[x.name]))
ax.legend(loc='upper center')
plt.show()

## 2 - Comparing means of two groups

For now, let's look at only types 4 and 5 (supergiant and hypergiant). These are of particular interest to your supervisor, Dr Howe.

In [ ]:
types = [4,5]

sample = data.luminosity.apply(np.log)
grouped = sample.groupby(data.type)

xlab = 'log(luminosity)'
ylab = 'freq'

displayed = pd.concat([grouped.get_group(t) for t in types])
bins = np.linspace(displayed.min(), displayed.max(), 20)
ax = plt.axes()
ax.set_xlabel(xlab)
ax.set_ylabel(ylab)

plt.hist(grouped.get_group(types[0]), bins, alpha=0.5, label=type_key[types[0]], color='C' + str(types[0]))
plt.hist(grouped.get_group(types[1]), bins, alpha=0.5, label=type_key[types[1]], color='C' + str(types[1]))

ax.legend(loc='upper left')
plt.show()

Dr Howe has noticed that supergiants and hypergiants seem to have very similar luminosity distributions. She asks you to check whether they have the same mean.

### Question: do types 4 and 5 have the same mean luminosity?

The sample means of log(luminosity) are easy to obtain:

In [ ]:
type4 = data[data.type == 4].luminosity.apply(np.log)
type5 = data[data.type == 5].luminosity.apply(np.log)

mean4 = type4.mean()
mean5 = type5.mean()

print('Type 4:', mean4)
print('Type 5:', mean5)
print('difference:', mean4 - mean5)

They are certainly very similar, but is the difference between them statistically significant?

<br>

From the histogram, both distributions of log(luminosity) seem approximately symmetrical and with a rough bell-curve, so for now we will assume that they are normally distributed. (We will look later at how to test for normality.)

We can therefore choose a **parametric test** for the difference between two means. This means that the test uses a defined probability distribution (e.g. the normal distribution) as a model for the process that generates the data.

<br>

In general, if the assumptions of a parametric test are satisfied then it will provide more **statistical power** than a non-parametric alternative. Statistical power is defined as the probability that the test *correctly rejects the null hypothesis when it is false*, also known as its *sensitivity* or *true positive rate*.

Different parametric test make different **assumptions** about the data, so it is important to think carefully about whether these are satisfied before deciding on a particular test.


In this example, a [*t-test*](https://en.wikipedia.org/wiki/Student%27s_t-test) is appropriate:

### t-test for 2 independent groups

#### Theory


When comparing two samples (1 and 2), we will refer to their sizes as $n_1$ and $n_2$, their sample means as $\bar{x}_1$ and $\bar{x}_2$ and their sample standard deviations as $s_1$ and $s_2$.

Recall that 

$$\bar{x} = \frac{\sum_{i=1}^n x_i}{n}$$

is the *sample mean*

and 

$$s^2 = \frac{\sum_{i=1}^n (x_i - \bar{x})^2}{n-1}$$

is the *unbiased sample variance*.

<br>

For our example, we need a two-tailed test:

$H_0$: The two samples come from the same distribution with mean $\mu = \mu_1 = \mu_2$.

$H_1$: The samples come from two different distributions, with means $\mu_1 \ne \mu_2$.

<br>

The test statistic is given by

$$t = \frac{\bar {x}_1 - \bar{x}_2}{s_p \cdot \sqrt{\frac{1}{n_1}+\frac{1}{n_2}}}$$,

where 

$$s_p^2 = \frac{\left(n_1-1\right)s_1^2 + \left(n_2-1\right)s_2^2}{n_1 + n_2-2}$$ 

is an unbiased estimator of the *pooled variance* of the two samples.

<br>

Under $H_0$, the test statistic $t$ follows a *Student's t-distribution* with $n_1 + n_2 - 2$ degrees of freedom.

We use this distribution to calculate a p-value for the observed value of the test statistic, $t$.


#### Assumptions

- The means of the two samples follow normal distributions. This is true if the samples themselves are normal, but also true for any other distribution if $n$ is large (by the *central limit theorem*).
- The two populations have equal variance.
- The two samples are independent.

Two-sample t-tests are robust to moderate deviations from these assumptions, but major deviations may produce misleading results.

#### Application

$H_0$: $\mu_{\text{type4}} = \mu_{\text{type5}}$.

$H_1$: $\mu_{\text{type4}} \ne \mu_{\text{type5}}$.

Let's set a significance level $\alpha=0.05$

Using a statistical library such as `scipy.stats`, we just supply the data for each sample and the test function deals with the rest:

In [ ]:
stats.ttest_ind(type4, type5)

We can visualise this result on the t-distribution:

In [ ]:
t_obs = stats.ttest_ind(type4, type5).statistic
df = len(type4) + len(type5) - 1
print("degrees of freedom:", df)

tmin = -4
tmax = 4

t = stats.t(df)
x = np.linspace(tmin,tmax,100)
ax = plt.axes()
ax.set_xlabel("t")
ax.set_ylabel("pdf")
plt.plot(x, t.pdf(x), color='gray')

# the area of the shaded region is the two-tailed p-value
lower_tail = np.linspace(tmin,-t_obs,100)
upper_tail = np.linspace(t_obs,tmax,100)
plt.fill_between(lower_tail,t.pdf(lower_tail),color='lightgrey')
plt.fill_between(upper_tail,t.pdf(upper_tail),color='lightgrey')
plt.show()

The t-test p-value is greater than than $\alpha$, so we accept the null hypothesis that the means are equal.

### Other types of t-test

#### One-tailed t-test

In the example above, we used a *two-tailed test* (because $H_1:\mu_1 \ne \mu_2$ was symmetrical). 

For a *one-tailed test*, we need to halve the two-sided p-value, e.g. 

$H_1:\mu_{\text{type4}}>\mu_{\text{type5}}$ would give us 

In [ ]:
ax = plt.axes()
ax.set_xlabel("t")
ax.set_ylabel("pdf")
plt.plot(x, t.pdf(x), color='gray')

# the area of the shaded region is the one-tailed p-value for H1: mu_1 > mu_2
upper_tail = np.linspace(t_obs,tmax,100)
plt.fill_between(upper_tail,t.pdf(upper_tail),color='lightgrey')
plt.show()

In [ ]:
p_2tailed = stats.ttest_ind(type4, type5).pvalue
p_1tailed = p_2tailed/2
print('1-tailed p-value:', p_1tailed)

To test the complementary hypothesis (i.e. $H_1:\mu_1<\mu_2$), we would need to use `1 - p_1tailed`, e.g. 

$H_1:\mu_{\text{type4}}<\mu_{\text{type5}}$ would give us

In [ ]:
ax = plt.axes()
ax.set_xlabel("t")
ax.set_ylabel("pdf")
plt.plot(x, t.pdf(x), color='gray')

# the area of the shaded region is the one-tailed p-value for H1: mu_1 < mu_2
lower_tail = np.linspace(tmin,t_obs,100)
plt.fill_between(lower_tail,t.pdf(lower_tail),color='lightgrey')
plt.show()

In [ ]:
p_2tailed = stats.ttest_ind(type5, type4).pvalue
p_1tailed = p_2tailed/2
print('1-tailed p-value:', 1 - p_1tailed)


#### Paired two-sample t-test

Sometimes we have two samples with paired observations (for example, luminosity of the same set of stars, as measured on two different dates). This situation requires testing whether the *mean of the differences* between pairs is zero, which is called a [*paired two-sample t-test*](https://en.wikipedia.org/wiki/Student%27s_t-test#Dependent_t-test_for_paired_samples).

#### Welch's t-test

If the sample sizes in the two groups being compared are equal, Student's original t-test is highly robust to the presence of unequal variances. [*Welch's t-test*](https://en.wikipedia.org/wiki/Welch%27s_t-test) is an alternative that is insensitive to equality of the variances, regardless of whether the sample sizes are similar.

#### One-sample t-test

For cases where we want to compare a sample against a theoretical mean, we can use the [*one-sample t-test*](https://en.wikipedia.org/wiki/Student%27s_t-test#One-sample_t-test).

### Alternatives to the t-test

#### Mann-Whitney U-test

For non-normal samples where $n$ is small, the assumptions of the t-test break down. However, we can use a *non-parametric test* to compare two samples, whatever the shape of their distributions.

The [*Mann-Whitney U-test*](https://en.wikipedia.org/wiki/Mann–Whitney_U_test) (aka Wilcoxon rank-sum test) is one such test. The null hypothesis for this test is that a randomly selected value from sample 1 is equally likely to be less than or greater than a randomly selected value from sample 2. If the distributions are sufficiently different, the resulting p-value will be small and we will reject this null hypothesis. Note that the U-test does not compare the sample means directly.


#### Wilcoxon signed-rank test

The [*Wilcoxon signed-rank test*](https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test) is is the paired-sample version of the Mann-Whitney U-test.

<br>


## 3 - Comparing variances of two groups

The t-test only compares the *means* of the two groups. 
Next, Dr Howe would like you to check their *variances*.

### Question: do types 4 and 5 have the same variance in log(luminosity)?

### Q-Q plot

The [*quantile-quantile plot*](https://en.wikipedia.org/wiki/Q–Q_plot) (Q-Q plot) is a simple, graphical method to check whether two sets of observations appear to come from the same distribution, or to compare one set of data to a theoretical distribution.

It is made by plotting the quantiles (i.e. percentiles) of the two distributions against each other.

If the variances are the same, the Q-Q plot will approximate a straight line with gradient 1.

We can find the percentiles for our sample directly with `pandas`:

In [ ]:
x = np.linspace(0,1,101)
t4q = np.array(type4.quantile(x))
t5q = np.array(type5.quantile(x))

f = plt.figure(figsize=(12, 4))

ymin=11
ymax=14

ax = plt.subplot(121)
ax.set_xlabel('quantile')
ax.set_ylabel('type 4')
ax.set_ylim([ymin,ymax])
plt.scatter(x,t4q,color='C4')

ax = plt.subplot(122)
ax.set_xlabel('quantile')
ax.set_ylabel('type 5')
ax.set_ylim([ymin,ymax])
plt.scatter(x,t5q,color='C5')

plt.show()


Plotting type 5 against type 4:

In [ ]:
xlab = 'type 4'
ylab = 'type 5'

ax = plt.axes()
ax.set_xlabel(xlab)
ax.set_ylabel(ylab)

plt.scatter(t4q, t5q, color='gray')

# the line with gradient 1, passing through Q50:
m = 1
c = t5q[50] - t4q[50] * m
plt.plot(t4q, m * t4q + c, color='black')

plt.show()

This looks like a pretty good fit, but we can do an [*F-test*](https://en.wikipedia.org/wiki/F-test_of_equality_of_variances) to be more rigorous:

### F-test for equality of variances

#### Theory

Once again, we need a two-tailed test:

$H_0$: The two populations have identical variance  $\sigma^2 = \sigma_1^2 = \sigma_2^2$.

$H_1$: The two populations have non-identical variances, $\sigma_1^2 \ne \sigma_2^2$.

The test statistic is simply the ratio of the sample variances:

$$F = \frac{s_1^2}{s_2^2}$$.

Under $H_0$, $F$ follows an [*F-distribution*](https://en.wikipedia.org/wiki/F-distribution) with parameters $(n_1 - 1,n_2 - 1)$.

We use this distribution to calculate a p-value for the observed value of the test statistic, $F$.


#### Assumpions

- The two samples both follow normal distributions.

Note that the means of the two populations may differ.
The F-test is highly sensitive to deviations from the assumption of normality.


#### Application

We will set $\alpha=0.05$.

We can visualise the F-distribution corresponding to our example ($n_1 = n_2 = 40$):

In [ ]:
x = np.linspace(0.1,3)
plt.plot(x,stats.f.pdf(x,39,39), color='gray')
plt.show()

We will calculate the test statistic:

In [ ]:
fstat = type4.var()/type5.var()
print("F =",fstat)

plt.plot(x,stats.f.pdf(x,39,39), color='gray')
x_region = np.linspace(0.1,fstat,100)
plt.fill_between(x_region,stats.f.pdf(x_region,39,39),color='lightgrey')
plt.show()

We use the CDF to calculate the left-tail $p$-value and double it for a two-tailed test:

In [ ]:
p_value = stats.f.cdf(fstat,39,39) * 2
print("p =",p_value)

Here, $p>\alpha$ so we accept the null hypothesis of equal variance, at the 5% level.

<br>

## 4 - Comparing means of more than two groups

During the course of your investigations for Dr Howe, you have noticed that the distributions of the dwarf stars' luminosities (types 0,1 and 2) are also overlapping.

In [ ]:
types = [0,1,2]

sample = data.luminosity.apply(np.log)
grouped = sample.groupby(data.type)

xlab = 'log(luminosity)'
ylab = 'freq'

displayed = pd.concat([grouped.get_group(t) for t in types])
bins = np.linspace(displayed.min(), displayed.max(), 20)
ax = plt.axes()
ax.set_xlabel(xlab)
ax.set_ylabel(ylab)

for t in types:
    plt.hist(grouped.get_group(t), bins, alpha=0.5, label=type_key[t], color='C' + str(t))

ax.legend(loc='upper right')
plt.show()

### Question: do types 0, 1 and 2 have the same mean luminosity?

The t-test can only compare the means of two samples (or one sample with a theoretical mean). 

To move beyond two samples, we need to use a different method called [*analysis of variance*](https://en.wikipedia.org/wiki/Analysis_of_variance) (ANOVA).

### One-way ANOVA

#### Theory

$H_0$: All of the groups have identical means:  $\mu = \mu_1 = \mu_2 = \mu_3$.

$H_1$: Not all of the group means are identical.

The one-way (also known as single-factor) ANOVA uses the F-test to compare the within-group and between-group variation:

$$F = \frac{\text{between-group variation}}{\text{within-group variation}}$$

Under $H_0$, $F$ follows an F-distribution with parameters $(g-1,n_T-1)$, where $g$ is the number of groups (here, 3 types of star), and $n_T$ is the total number of observations.

Once again, the F-distribution provides a p-value associated with the calculated value of $F$.

#### Assumptions

- Observations are independent.
- Populations are normally distributed.
- Variances of the populations are equal.



#### Application

We will set $\alpha=0.05$.

In [ ]:
stats.f_oneway(grouped.get_group(0),grouped.get_group(1),grouped.get_group(2))

Here we have $p<\alpha$, so we reject $H_0$: the three groups do not appear to have the same mean luminosity.

### Other types of ANOVA

ANOVA is an important element of statistical analysis when we are interested in comparing the effects of different treatments. 

The underlying statistical model changes, depending on the expected relationship between treatment and effect (*fixed-*, *random-* or *mixed-effects*).

Where multiple variables change simultaneously (for example, in patient populations), we may need to consider *multiple factors* (e.g. *two-way ANOVA*) and the *interactions* between factors.

<br>

## 5 - Comparing distributions over a categorical variable

Your undergraduate project student, Tunde, is looking at the distributions of spectral class (i.e. colour) for various types of star. 

He says that the bar charts for white dwarves (type 2) and main sequence stars (type 3) seem similar, but you are not so sure. 


In [ ]:
labels = ['O','B','A','F','G','K','M']

grouped = data.groupby(data.type)

classes2 = grouped.get_group(2).spectral_class.value_counts()
classes3 = grouped.get_group(3).spectral_class.value_counts()

for i in labels:
    if (i not in classes2):
        classes2[i] = 0 
    if (i not in classes3):
        classes3[i] = 0 

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, [classes2[i] for i in labels], width, alpha=0.4, color = 'C2', label=type_key[2])
rects2 = ax.bar(x + width/2, [classes3[i] for i in labels], width, alpha=0.8, color = 'C3', label=type_key[3])
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend(loc='upper right')

plt.show()


### Question: do types 2 and 3 share the same distribution of spectral class?

To test differences in distributions over a *categorical variable* like spectral class, we can use [*Pearson's chi-squared test*](https://en.wikipedia.org/wiki/Chi-squared_test):

### Chi-squared test for statistical independence

#### Theory

$H_0$: Probability distribution for the categorical variable (e.g. spectral class) is independent of group (e.g. star type).

$H_1$: Probability distribution depends on group.

We need to find out whether the differences in observed frequencies between the two groups are small enough to have arisen by chance. We do this by constructing a [*contingency table*](https://en.wikipedia.org/wiki/Contingency_table) showing the observed frequency of each outcome for each of the two groups, and comparing to the *expected frequencies* under $H_0$.


The test statistic is

$$X^2 = \sum^k_{i=1}{\frac{(x_i-m_i)^2}{m_i}}$$,

where $k$ is the number of cells in the table and $x_i$ and $m_i$ are the observed and expected frequencies for each cell.

Under $H_0$, $X^2$ follows a [$\chi^2$ distribution](https://en.wikipedia.org/wiki/Chi-squared_distribution), which is parametrised by the number of degrees of freedom. A contingency table of size $a \times b$ has $(a-1)(b-1)$ degrees of freedom, i.e. the number of independent counts when row and column sums are held fixed.

#### Assumptions

- Sampling is random
- Each sample is independent of the others
- Expected frequency for each cell must be sufficiently large

The approximation to the $\chi^2$ distribution breaks down if expected frequencies are too low. It will normally be acceptable so long as no more than 20% of the events have expected frequencies below 5. Where there is only 1 degree of freedom (i.e. a 2 $\times$ 2 table), the approximation is not reliable if expected frequencies fall below 10. 


#### Application

We will set $\alpha=0.05$.

We begin by making a contingency table for the observed spectral classes.

In [ ]:
# OBSERVED

types = [2,3]
subset = pd.concat([grouped.get_group(t) for t in types])

obs = pd.crosstab(subset.type,subset.spectral_class,margins=True,margins_name="total")
print("OBSERVED:")
obs

From the row and column totals, we can calculate the *expected* frequencies under $H_0$:

In [ ]:
# EXPECTED

exp = pd.DataFrame(stats.contingency.expected_freq(obs))
exp.columns = obs.columns
exp = exp.rename(index={0:2,1:3,2:"total"})
exp = exp.rename_axis(columns=obs.axes[1].name,index=obs.axes[0].name)
print("EXPECTED:")
exp

The expected values for classes K and O are too small. We can combine the last three columns to get all of the expected values over 5:

In [ ]:
# OBSERVED (combining columns)

new_obs = obs[['A','B']][:-1]
new_obs['others'] = obs[['F','K','O']][:-1].sum(axis=1)
print("OBSERVED:")
new_obs

In [ ]:
# EXPECTED (combining columns)

new_exp = pd.DataFrame(stats.contingency.expected_freq(new_obs))
new_exp.columns = new_obs.columns
new_exp = new_exp.rename(index={0:2,1:3})
new_exp = new_exp.rename_axis(columns=new_obs.axes[1].name,index=new_obs.axes[0].name)
print("EXPECTED:")
new_exp

In `scipy.stats`, he most convenient form of the chi-squared test for a contingency table is `chi2_contingency`, which just takes the table of observations as input and calculates the expected values and degrees of freedom accordingly.

In [ ]:
p_value = stats.chi2_contingency(new_obs)[1]
p_value

According to the chi-squared test, $p<\alpha$, so there is enough evidence to reject $H_0$:
it appears that white dwarves and main sequence stars follow different distributions for spectral class.

<br>

## 6 - Testing for normality

Professor Xu thinks that the current system for classifying stars can be improved. In particular, she thinks that log(temperature) should be normally distributed for each star type. 

She has asked you to find out whether this is true under the current classification system, and if not, which types should be revised.

### Question: for which star types is log(temperature) normally distributed?

### Q-Q plot

#### Theory

We can use a Q-Q plot to investigate whether each sample resembles a normal distribution.

If we set

$x =$ the theoretical quantiles from the standard normal ($Z$) distribution, and

$y =$ the quantiles from the sample,

the Q-Q plot will be close to a straight line for samples that are approximately normally distributed.

Although this is not a rigorous statistical method, it can be enough to suggest whether a normal approximation is likely to be valid for a particular data set, or to diagnose [*skewness*](https://en.wikipedia.org/wiki/Skewness) and/or [*kurtosis*](https://en.wikipedia.org/wiki/Kurtosis) :

In [ ]:
def do_plots(sample, col='gray', lab=None, mu=None, sigma=None):
    
    if(mu):
        n = len(sample)
        norm_fit = stats.norm(loc=mu, scale=sigma)
  
    f = plt.figure(figsize=(12, 4))
    
    # histogram
    ax = plt.subplot(121)
    
    nbins = 20
    smin = sample.min()
    smax = sample.max()
    binwidth = (smax - smin)/nbins
    bins = np.linspace(sample.min(), sample.max(), nbins)
    
    xlab = 'observed'
    ylab = 'freq'
    ax.set_xlabel(xlab)
    ax.set_ylabel(ylab)
    
    ax.hist(sample, bins, alpha=0.5, color=col, label=lab)
    
    if(mu):
        plt.plot(bins, n * binwidth * norm_fit.pdf(bins), color='black')
    
    if(lab):
        ax.legend(loc='upper left')
    
    # Q-Q plot
    
    ax = plt.subplot(122)
    
    x = np.linspace(0,1,100)
    normal_q = stats.norm.ppf(x)
    sample_q = np.array(np.quantile(sample,x))
    
    xlab = 'standard normal'
    ylab = 'observed'
    ax.set_xlabel(xlab)
    ax.set_ylabel(ylab)
    
    # the plot itself:
    ax.scatter(normal_q,sample_q, color=col, label=lab)
    
    # the line passing through Q25 and Q75:
    m = (sample_q[75] - sample_q[25])/(normal_q[75] - normal_q[25])
    c = sample_q[25] - normal_q[25] * m
    ax.plot(normal_q, m * normal_q + c, color='black')
    
    if(lab):
        ax.legend(loc='upper left')
    
    plt.show()

In [ ]:
### normally distributed data
sample = stats.norm.rvs(loc=10,scale=2,size=100)

### positively skewed data (e.g. lognormal)
#sample = stats.lognorm.rvs(0.5,size=100)

### negatively skewed data (e.g. constant - lognormal)
#sample = 20 - stats.lognorm.rvs(0.5,size=100)

### leptokurtic (heavy-tailed) data (e.g. Student's t-distribution with df=2)
#sample = stats.t.rvs(2,loc=200,size=100)

### platykurtic (thin-tailed) data (e.g. uniform distribution)
#sample = stats.uniform.rvs(loc=10,scale=20,size=100)

do_plots(sample)

It can also be a useful way to identify the data points that are responsible for any deviations from normality.

#### Application

Run the code below to see the histograms and Q-Q plots for log(temperature) for each star type:

In [ ]:
### Set the star type to test
t = 0

sample = data[data.type == t].temperature.apply(np.log)
col= 'C' + str(t)
lab= type_key[t]

mu = sample.mean()
sigma = sample.std()

do_plots(sample,col,lab,mu,sigma)
print(type_key[t])
print('mean:', mu, '   SD:', sigma)

For which of the star types does log(temperature) appear approximately normal? How would you describe the other distributions?

Of course, for a more rigorous investigation of normality, we can use a statistical test:

### Shapiro-Wilk test

#### Theory

The [*Shapiro–Wilk test*](https://en.wikipedia.org/wiki/Shapiro–Wilk_test) tests the null hypothesis that a sample $x_1, ..., x_n$ came from a normally distributed population.

It compares statistics obtained from the observed data to the expected values of statistics sampled from the standard normal distribution.

#### Application

The Shapiro-Wilk test is easy to apply in scipy:

In [ ]:
### set the star type to test
t = 0

sample = data[data.type == t].temperature.apply(np.log)
p_value = stats.shapiro(sample)[1]

print(type_key[t])
print('p =', p_value)

Apply the Shapiro-Wilk test to each of the star types. Which of them produce p-values less than $\alpha$? 

### Alternative tests for normality

Many other tests for normality have been developed, including the Anderson-Darling, Cramér–von Mises and Kolmogorov-Smirnov (see below) tests. 

The Shapiro-Wilk test has been found to have the best statistical power for a given significance level.

<br>

## 7 - Correcting for multiple hypothesis tests

Unfortunately, there is a problem with the previous analysis.

Recall that the significance level, $\alpha$, is defined as the probability of incorrectly rejecting $H_0$ when it is actually true (i.e. the probability of a Type I error).

When we perform [*multiple related hypothesis tests*](https://en.wikipedia.org/wiki/Multiple_comparisons_problem), we increase the chances of producing such a Type I error.

For example, if $\alpha=0.05$ and we perform 100 tests, we *expect* to generate 5 Type I errors. This can be a serious problem when large numbers of hypothesis tests are carried out simultaneously, for example in screening thousands of genes for association with a disease.

We therefore need a strategy to control the rate of Type I errors. A very simple approach is given by the [*Bonferroni correction*](https://en.wikipedia.org/wiki/Bonferroni_correction):

### Bonferroni correction

#### Theory

When conducting $n$ related hypothesis tests, we reduce the significance level for each test to $\alpha/n$.

The probability of making a Type I error *over the whole set of tests* (known as the *family-wise error rate*, FWER) therefore remains at $\alpha$.

#### Application

In [ ]:
print("Shapiro-Wilk test for normality\n")

p_values = []
alpha = 0.05
n = 6

for t in range(0,n):
    sample = data[data.type == t].temperature.apply(np.log)
    p_values.append(stats.shapiro(sample)[1])

print("with uncorrected alpha =",np.format_float_scientific(alpha,3),":")
for i in range(0,n):
    result = ""
    if(p_values[i] < alpha): result = "*** REJECT H0 ***"
    print(i, type_key[i], ": p =", np.format_float_scientific(p_values[i],3), result) 


    
print("\nwith Bonferroni correction, alpha/n =",np.format_float_scientific(alpha/n,3),":")
for i in range(0,n):
    result = ""
    if(p_values[i] < alpha/n): result = "*** REJECT H0 ***"
    print(i, type_key[i], ": p =", np.format_float_scientific(p_values[i],3), result) 


    


After correcting for multiple hypothesis testing, the red dwarf p-value is not significant.

We should report to Professor Xu that log(temperature) is not normally distributed for the brown dwarf, supergiant and hypergiant types.


### Alternative methods for multiple testing correction

The Bonferroni correction is simple to apply, but it may be too conservative when there is a very large numbers of tests, or when the tests are not independent (for example, genes are often related to other genes so are likely to share properties).

The [*Benjamini-Hochberg procedure*](https://en.wikipedia.org/wiki/False_discovery_rate#Benjamini–Hochberg_procedure) is an alternative approach. Instead of controlling the FWER, this method controls the *proportion of the positive tests that are incorrect*, i.e. the proportion of rejected $H_0$'s that are Type I errors. This is known as the *false-discovery rate*, FDR.

<br>

## 8 - Goodness of fit

Your colleague Althea is not a fan of Prof. Xu's temperature scheme. She has her own ideas about how the star classification should be revised.

According to Althea's new theory of stellar evolution, there are two subtypes of hypergiant star, with very different temperature distributions. 

She says that our galaxy has approximately equal numbers of each subtype. 

If this theory is true, hypergiant temperatures should be distributed according to:

$$
P(3000K \leq \text{T} \lt 4000K) = 0.5 \\
P(4000K \leq \text{T} \lt 40000K) = 0.5
$$

with uniform temperature distributions within each of the two subtypes.

Althea asks you to check whether her theory agrees with your data set.

### Question: do hypergiants in the observed data set fit Althea's theory?

The pdf associated with the theory is piecewise uniform. We can construct this as a bespoke continuous probability distribution with `scipy.stats`:

In [ ]:
def althea_F(x):
    if(x<3000):
        return 0
    elif(x<4000):
        return 0.5*(x-3000)/(4000-3000)
    elif(x<40000):
        return 0.5 + 0.5*(x-4000)/(40000-4000)
    else:
        return 1

class althea(stats.rv_continuous):
    "Althea's distribution"
    def _cdf(self, x):
        return np.vectorize(althea_F)(x)

In [ ]:
al = althea()
x = np.arange(0,50000,100)

y = []
for z in x:
    y.append(al.pdf(z))

xlab = 'temperature'
ylab = 'probability density'    

ax = plt.axes()
ax.set_xlabel(xlab)
ax.set_ylabel(ylab)
plt.plot(x, y, color='tab:orange', label='theoretical pdf')
ax.legend(loc='upper right')
plt.show()    


At first glance, the histogram for the hypergiants (type 5) does appear to have a similar shape to this pdf:

In [ ]:
observed = data[data.type == 5].temperature
n = len(observed)

xlab = 'temperature'
ylab = 'freq'

bins = np.linspace(0,50000)
ax = plt.axes()
ax.set_xlabel(xlab)
ax.set_ylabel(ylab)
obs_hist = plt.hist(observed, bins, alpha=0.5, label='observed data')
ax.legend(loc='upper right')
plt.show()

To see more clealy, we can overlay a random sample of the same size ($n$=40), drawn from the theoretical distribution:

In [ ]:
expected = al.rvs(size=n)

xlab = 'temperature'
ylab = 'freq'

bins = np.linspace(0,50000)
ax = plt.axes()
ax.set_xlabel(xlab)
ax.set_ylabel(ylab)
obs_hist = plt.hist(observed, bins, alpha=0.5, label='observed data')
exp_hist = plt.hist(expected, bins, alpha=0.5, label='sampled from theoretical distribution')
ax.legend(loc='upper right')
plt.show()

How can we test more formally whether the observed data appear to be drawn from the theoretical distribution?

Here we have a theoretical distribution *which is not directly related to any of the standard statistical distributions*, so parametric methods such as the Shapiro-Wilk test are not applicable.

However, we can still perform a *non-parametric* goodness-of-fit test, by comparing the theoretical cdf with the empirical one:

In [ ]:
hist = np.histogram(observed, bins=100)
empirical_dist = stats.rv_histogram(hist)

x = np.arange(0,50000,100)

y_theoretical = []
y_empirical = []

for z in x:
    y_theoretical.append(al.cdf(z))
    y_empirical.append(empirical_dist.cdf(z))

xlab = 'temperature'
ylab = 'cumulative probability'
    
ax = plt.axes()
ax.set_xlabel(xlab)
ax.set_ylabel(ylab)
plt.plot(x, y_empirical, label='empirical cdf')
plt.plot(x, y_theoretical, label='theoretical cdf')
plt.legend(loc='lower right')
plt.show()

### Kolmogorov-Smirnov test

#### Theory

The [*Kolmogorov-Smirnov test*](https://en.wikipedia.org/wiki/Kolmogorov–Smirnov_test) (or K-S test) examines the deviation of the empirical cdf, $F_n(x)$, from the theoretical one, $F(x)$, to assess the goodness-of-fit. The test statistic is the quantity

$$
D_n= \sup_x |F_n(x)-F(x)|,
$$

i.e. the greatest vertical distance between the two curves.

The distribution of $D_n$ under the null hypothesis $H_0: F_n(x) = F(x)$ is called the *Kolmogorov distribution*.

#### Application

In our example,

$H_0$: $F_n(x) = F(x)$  : The observed temperature distribution of hypergiants is described by Althea's theory.

$H_1$: $F_n(x) \ne F(x)$  : The observed temperature distribution of hypergiants is not described by Althea's theory.

$\alpha = 0.05$

Graphically, we can plot $D_n$ for each $x$-value in the plot above:

In [ ]:
xlab = 'temperature'
ylab = 'empirical cdf - theoretical cdf'

#y_values = np.abs(np.array(y_empirical)-np.array(y_theoretical))
y_values = np.array(y_empirical)-np.array(y_theoretical)
    
ax = plt.axes()
ax.set_xlabel(xlab)
ax.set_ylabel(ylab)
plt.plot(x, y_values, color='black')
plt.show()

D_n = np.max(np.abs(y_values))
print('D_n:', D_n)

The test itself is easy in `scipy.stats`:

In [ ]:
result = stats.kstest(observed, al.cdf)
print(result)

The resulting $p > \alpha$, so we accept the null hypothesis: the observed data appear to be compatible with Althea's theory.

### Other applications of the K-S test

#### Goodness of fit

The K-S test can be used as an alternative method for testing normality, or as a goodness-of-fit test for any other theoretical distribution. However, the standard test is only valid if the parameters (e.g. mean and variance) are *not* estimated from the data. 

If parameters *are* estimated from the data, we will need to use simulation to find an empirical distribution for $D_n$ under $H_0$.


#### Two sample test

The K-S test is often used to compare two samples (in the absence of a theoretical cdf) to test whether they follow the same distribution. 

In `scipy.stats`, this is done with the function `ks_2samp`.